In [1]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import settrade_v2
from settrade_v2 import Investor
import time

In [2]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [3]:
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")
print("📂 Keyspaces:")
for row in rows:
    print("-", row.keyspace_name)

📂 Keyspaces:
- system_auth
- system_schema
- system_distributed
- system
- settrade_api
- system_traces


In [6]:
keyspace = "settrade_api"   # 👈 แก้เป็น keyspace ของคุณ
rows = session.execute(f"SELECT table_name FROM system_schema.tables WHERE keyspace_name='{keyspace}';")
print(f"\n📑 Tables in keyspace '{keyspace}':")
for row in rows:
    print("-", row.table_name)


📑 Tables in keyspace 'settrade_api':
- candlestick_data


In [7]:
table = "candlestick_data"
rows = session.execute(f"SELECT column_name, kind, type FROM system_schema.columns WHERE keyspace_name='{keyspace}' AND table_name='{table}';")
print(f"\n📊 Columns in {keyspace}.{table}:")
for row in rows:
    print(f"- {row.column_name} ({row.kind}): {row.type}")


📊 Columns in settrade_api.candlestick_data:
- close_price (regular): float
- high_price (regular): float
- low_price (regular): float
- open_price (regular): float
- symbol (regular): text
- time (partition_key): timestamp
- value (regular): float
- volume (regular): int


In [9]:
session.set_keyspace('settrade_api')

In [17]:
rows = session.execute("""
    SELECT * FROM candlestick_data WHERE symbol = 'JKN' ALLOW FILTERING;
""")

In [18]:
df = pd.DataFrame(rows, columns=[
    'symbol', 'time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'value'
])

# ✅ แปลงชนิดเวลา + เรียงลำดับ
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)

In [19]:
df.head()

,symbol,time,open_price,high_price,low_price,close_price,volume,value
0,2024-09-12,1970-01-01,0.31,0.26,0.30,JKN,7673886.0,27529083
1,2024-09-06,1970-01-01,0.35,0.32,0.34,JKN,6682106.0,20066358
2,2024-09-05,1970-01-01,0.37,0.33,0.37,JKN,6593577.5,19003496
3,2024-09-13,1970-01-01,0.29,0.27,0.28,JKN,4598449.0,16862624
4,2024-09-03,1970-01-01,0.44,0.37,0.38,JKN,24323800.0,60150728
